# The Ultimate Algo Trader (Neurobot 1.0)

![Python Trading bot](https://futurewithtech.com/wp-content/uploads/2021/06/Forex-trading-robots-scaled-1.jpg)

## Introduction:
This jupyter notebook contains code for developing and testing a dynamic algorithmic trader based on Python and Machine learning strategy.

## Steps:
1. **Data preperation**: Access historical market data via Alpaca API and preprocess it for analysis.
2. **Strategy creation**: Code algorithmic trading strategy based on "TBD".
3. **Backtesting-Optimization**: Backtest the strategy using historical data and fine-tune parameters for better performance.
4. **Risk/Reward**: Calculate both the risk and the reward based on the entry price, position size, stop-loss and target price
5. **live Trading(Optional)**: Implement the strategy for paper trading on Alpaca.

**Tools and Libaries** 
- Python, Pandas, Numpy, "TBD"

#### Notes: 
- This notebook is for educational and experimental purposes only.

### Imports and Dependencies

In [ ]:
import os
import numpy as np
import random
import seaborn as sns
import pandas as pd
import hvplot.pandas
import matplotlib.pyplot as plt
import alpaca_trade_api as tradeapi
from dotenv import load_dotenv
from scipy.interpolate import interp1d
from alpaca_trade_api.rest import REST, TimeFrame

# Initialize python files to import functions
import stock_data as data
import algo_strategy as strategy
import nn_models as model
import backtesting as backtest

import warnings
warnings.filterwarnings('ignore')

In [ ]:
load_dotenv()

### Data Processing and collection

In [ ]:
# import stock data
df = data.fetch_stock_data('2018-01-12', '2024-03-14', tickers = ['NVDA'], timeframe = '1Day')
df

In [ ]:
# data cleaning
df = df.rename(columns={'NVDA': 'NVDA close'})
df['Daily Returns'] = df['NVDA close'].pct_change()
df = df.dropna()
df

### Feauture Engineering - Time series Analysis

In [ ]:
df['Cumulative Returns'] = (1 + df['Daily Returns']).cumprod()
df

In [ ]:
df['Cumulative Returns'].plot()

In [ ]:
# set window sizes based on strategy
short_window = 5
long_window = 100

df['SMA_Fast'] = df['NVDA close'].rolling(window=short_window).mean()
df['SMA_Slow'] = df['NVDA close'].rolling(window=long_window).mean()

df = df.dropna()
df
# later on -> trial different training windows using DateOffset()

In [ ]:
# Initialize Signals based on strategy (i.e. DMAC, buy and hold, etc) 

In [ ]:
df['Daily Returns Lagged'] = df['Daily Returns'].shift(-1)
df

In [ ]:
df[['Daily Returns Lagged', 'Daily Returns']].corr()

# maybe not to use lagged strategy 
# consider DMAC? 

### Algo trading strategies - logic to comapre or combine trading strategy such as Daily returns lag, Anual returns, SMA, Exponential

In [ ]:
# Initialize a simple winnig trading strategy 
def simple_strategies(strategies, stock_data):
    
# Initialize a hybrid trading strategy    
def hybrid_strategies(Stock_data):
    """
    This function combine elements from multiple trading strategies to create a hybrid strategy
    
    Input: 
    - It uses historical stock data for evaluation, Stock_data.
    Returns:
    - It returns a hybrid_startegy trading signall
    """
    signal_one = #strategy_one(stock_data)
    signal_two
    
    
    
    
    

### Machine Learning (LSTM ) + other model logreg or SVM - Logic that compares which model 

### Backtesting - Model and strategy tuning logic to increase profitability

### Risk management and Rewards

### Fundamental Analysis (Predictions and Plotting)

In [ ]:
# From Algo Trading 3 live 

# Submit order
api.submit_order(
    symbol="META", 
    qty=number_of_shares, 
    side=orderSide, 
    time_in_force="gtc", 
    type="limit", 
    limit_price=limit_amount
